Machine Learning - Homework 1 (due Sep. 13)

Problem 1: Legal reasoning (from Murphy 2.2).

Suppose a crime has been committed. Blood is found at the scene for which there is no innocent explanation. It is of a type which is present in 1% of the population.

a. The prosecutor claims: “There is a 1% chance that the defendant would have the crime blood type if he
were innocent. Thus there is a 99% chance that he guilty”. This is known as the prosecutor’s fallacy.
What is wrong with this argument?

Using the following acronyms we can prove why the prosecutor is wrong.
I = innocence
G = Guilt
E = Evidence
Then P(E|I) represents the probability that an innocent suspect matches the evidence resulting in a false positive. P(I|E) is the probability that the defendant who matches the evidence is innocent.
Here the prosecutor is claiming that P(I|E) = P(E|I) = 1/100
but this is not correct since the value of P(I|E) = (P(E|I) * P(I))/P(E)
To get the value of P(E) we must use Bayes rule so P(E) = P(E|I) * P(I) + P(E|G) * P(G).
Here we need to know that P(I) = .99, P(G) = .01, P(E|G) = 1. So P(E) = .01 * .99 + .01 * 1 which means P(E) = .0199.

So using our P(E) in our Bayes fourmula we get P(I|E) = (.01 * .99)/.0199 meaning that P(I|E) is approximately .497.
Thus the probability that a person matching the evidence is guilty in this case is P(G|E) = 1 - P(I|E) = 50.3%

b. The defender claims: “The crime occurred in a city of 800,000 people. The blood type would be
found in approximately 8000 people. The evidence has provided a probability of just 1 in 8000 that
the defendant is guilty, and thus has no relevance.” This is known as the defender’s fallacy. What is
wrong with this argument?

Similar to the acronyms used in the previous question
I = innocence
G = Guilt
E = Evidence
Here P(G|E) will represent the probability that the defendant who matches the evidence is guilty. As proven previously P(G|E) is approximately 50.3% or 1/2
The misquote conviniently disguises the fact that the blood type combined with other evidence would likely prove the defendants guilt.

c. Suppose that forensic analysis tells us that with 98% probability, the blood found at the scene belongs to the perpetrator.  Given the information presented in the above two questions, determine the posterior probability the the defendent is guilty, given that the defendent's blood type matches that found at the crime scene *and* that the defendent was one of only 5 people with access to the crime scene.  

Our posterior probability should be the result of P(G|E) = (P(G)P(E|G))/P(E)
The acronyms here will be following the same acronyms above.
Here we need to recalculate some things due to the evidence that there is a pool of 5 possible suspects now
p(E) = P(E|I) * P(I) + P(E|G) * P(G)
we know that P(E|G) = 1, P(E|I) = .01 since there is still only a 1 in 100 chance that the blood matches and idividual. P(G) = .2 since there is a 1/5 possibility of being guilty thus P(I) = .8
so P(E) = .01 * .8 + 1 * .2 = .208
so out posterior P(G|E) = (P(G)P(E|G))/P(E)
is the caluation P(G|E) = ((.2)*1)/.208 = .9615 

so there is a 96.15% probability that the defendant is guilty given the blood match and the 5 potential suspects with access to the crime scene.

Problem 2: Naive Bayes.

One use of the naive Bayes classifier, which is still in practical use today, is as a spam filter.  Consider the corpus of text messages packaged with this homework, which are each labelled as either 'spam' or 'ham'.  In this case, naive Bayes utilizes a Bernoulli model that quantifies the probability of a given word given that the message is either spam or ham.  For example, investigating the text messages here, we find that the word *draw* shows up in spam 27 times, yet in ham only 5.  Thus, we have that
$$ P(X=\mathrm{draw}|Y=\mathrm{ham}) = \frac{5}{5+27}. $$

While this is not particularly strong evidence on its own, we can create a powerful classifier by using the naive assumption in conjunction with all the words in a given message:
$$ P(Y=\mathrm{ham}|\hat{X}) \propto P(Y=\mathrm{ham}) \prod_{i=1}^n P(X=x_i|Y=\mathrm{ham}), $$
$$ P(Y=\mathrm{spam}|\hat{X}) \propto P(Y=\mathrm{spam}) \prod_{i=1}^n P(X=x_i|Y=\mathrm{spam}), $$
where $x_i$ are the words in a given message. 

Your task is to write such a classifier.  I have taken the somewhat tedious step of parsing the data for you, yielding the variables *word_dictionary*, which contains the ham and spam counts for each word, as well as *training_labels*, which provides the spam/ham labels for each text message.  I have also parsed a set of test data: *test_messages* is a list, each entry containing another list of the words in the text message, as well as *test_labels* which contains the spam/ham label for each message.

In [2]:
import numpy as np

# Maps from 'ham' or 'spam' strings to zero or one
def mapper(s):
    if s=='spam':
        return 0
    else:
        return 1

# Read in the text file
f = open('SMSSpamCollection','r')
lines = f.readlines()

# Break out the test data
test_lines = lines[:len(lines)//5]
lines = lines[len(lines)//5:]

# Instantiate the frequency dictionary and an array to
# record whether the line is ham or spam
word_dictionary = {}
training_labels = np.zeros(len(lines),dtype=int)

# Loop over all the training messages
for i,l in enumerate(lines):
    # Split into words
    l = l.lower().split()
    # Record the special first word which always ham or spam
    if l[0]=='ham':
        training_labels[i] = 1
    # For each word in the message, record whether the message was ham or spam
    for w in l[1:]:
        # If we've never seen the word before, add a new dictionary entry
        if w not in word_dictionary:
            word_dictionary[w] = [1,1]
        word_dictionary[w][mapper(l[0])] += 1
        
# Loop over the test messages
test_labels = np.zeros(len(test_lines),dtype=int)
test_messages = []
for i,l in enumerate(test_lines):
    l = l.lower().split()
    if l[0]=='ham':
        test_labels[i] = 1
    test_messages.append(l)

Below, I have provided code skeletons.  Your job is to make the code skeletons into an operational naive Bayes spam detector.  (you may discard these skeletons if you would prefer to code this from scratch).  Note that lines where you will need to change the code are marked with a '#!'.

Your first task is train the model:  

In [4]:
#What is the prior P(Y=ham) ?
ham_prior = sum(training_labels)/len(training_labels) # Here the prior will be determined by the ham count over the total count of messages in the file

# What are the class probabilities P(X=word|Y=ham) for each word?
ham_likelihood = {}
for key,val in word_dictionary.items():
    ham_likelihood[key] = word_dictionary[key][1]/(word_dictionary[key][0]+word_dictionary[key][1]) #taking the number of ham counts and dividing by the totatl number of all counts

Your next task is to make predictions on a set of test examples which were held back from the training procedure (see *test_messages* variable).  For each of these messages, compute the ham and spam probabilities.

In [13]:
# Where to hold the ham and spam posteriors
posteriors = np.zeros((len(test_lines),2))

# Loop over all the messages in the test set
for i,m in enumerate(test_messages):
    posterior_ham = 1.0 
    posterior_spam = 1.0 
    spam_prior = 1 - ham_prior
    # Loop over all the words in each message
    for w in m:
        # If a word is not found in test_messages it is skipped
        try:
            posterior_ham *= (ham_prior * ham_likelihood[w])/(ham_likelihood[w] * ham_prior + spam_prior * (1-ham_likelihood[w]))
            posterior_spam *= (spam_prior * (1 - ham_likelihood[w]))/(ham_likelihood[w] * ham_prior + spam_prior * (1-ham_likelihood[w]))
        except KeyError:
            pass
    
    # Notice the normalization factor (denominator) 
    # to turn these into proper probabilities!
    posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
    posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)

Finally, make a ham/spam prediction based on your posterior probabilities.  Compare these to the labels contained in test_labels.  Report the accuracy of your classifier as percentage correct.

In [14]:
correct = 0
total = 0
for i in test_labels:
    if (posteriors[i,1] > posteriors[i,0])and(posteriors[i,1]==test_labels[i]):
        #print('one')
        correct += 1
    elif (posteriors[i,1] < posteriors[i,0])and(posteriors[i,0]==test_labels[i]):
        correct += 1
    total += 1

percentage = correct/total

#I have ran this code and multiple permutations of it in my own python script and I cannot for the life of me figure out why my percentage is so low
print('Percent correct: ', (percentage*100))
    

Percent correct:  15.080789946140035
